In [ ]:
from ltr.client import ElasticClient
client = ElasticClient()

In [ ]:
from ltr import download
download();

In [ ]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

In [ ]:
config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
                { #1
                    "name": "title_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"title": "{{keywords}}"}
                    }
                },
                { #2
                    "name": "overview_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"overview": "{{keywords}}"}
                    }
                }
            ]
    }}




from ltr import setup
setup(client, config=config, index='tmdb', featureset='movies')

In [ ]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/title_judgments.txt', 
                                        trainingOutFile='data/title_judgments_train.txt', 
                                        featureSet='movies')

In [ ]:
from ltr import train
trainLog  = train(client,
                  trainingInFile='data/title_judgments_train.txt',
                  index='tmdb',
                  featureSet='movies',
                  modelName='title')

In [ ]:
from ltr import search
search(client, "cartoon with basketball aliens", modelName='title')